### RNA Velocity with Salmon-Alevin/scVelo

#### As mentioned in the [Rmd overview](https://github.com/trichelab/single_cell_analyses/blob/master/rnavel_alevin_scvelo.Rmd) of this analysis pipeline, it proceeds along 4 main steps:

   1. [Installation of pre-requisite software components](#step1) <br>
   2. [Downloading and indexing the genome](#step2) <br>
   3. [Aligning your sequencing reads to intronic and exonic regions of the genome](#step3) <br>
   4. [Performing the RNA Velocity calculation](#step4) <br>

#### <a id='step1'>Step 1. Installation of pre-requisite software components</a>

#### <a id='step2'>Step 2. Downloading and indexing the reference genome</a>

Download reference genome and the corresponding gtf file. In this example, we are using Gencode Grch38 primary assembly 
> 
wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_34/GRCh38.primary_assembly.genome.fa.gz <br>
wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_34/gencode.v34.annotation.gtf.gz 

Now, it's time to open R studio and import (or install if you have not done so) packages required. The R script for indexing the reference is [here](https://github.com/trichelab/single_cell_analyses/Rscripts/step2_index_ref_genome.R)

#### <a id='step3'>Step 3. Align and quantify reads to intronic and exonic regions of the genome</a>

#### <a id='step4'>Step 4. Perform RNA velocity calculation with scVelo</a>